In [1]:
# Create coldstate.nc
# Creates empty `coldstate.nc` file for initial SUMMA runs. This can be replaced by a more elegant initial conditions file, such as generated by SUMMA's `-r y` (create a restart file, yearly intervals) command line option. 
#
# Note on HRU order
# HRU order must be the same in forcing, attributes, initial conditions and trial parameter files. Order will be taken from forcing files to ensure consistency.

# modules
import os
import numpy as np
import xarray as xr
import netCDF4 as nc4
from pathlib import Path
from shutil import copyfile
from datetime import datetime
from utils.read_files import read_from_control, make_default_path


In [2]:
# Store the name of the 'active' file in a variable
basin = "EastRiver"
controlFile = f'control_{basin}.txt'

In [3]:
# --- Find forcing location and an example file
# Forcing path
forcing_path = read_from_control(controlFile,'forcing_summa_path')

# Specify default path if needed
if forcing_path == 'default':
    forcing_path = make_default_path('forcing/4_SUMMA_input',controlFile) # outputs a Path()
else:
    forcing_path = Path(forcing_path) # make sure a user-specified path is a Path()
    
# Find a list of forcing files
_,_,forcing_files = next(os.walk(forcing_path))

# Select a random file as a template for hruId order
forcing_name = forcing_files[0]

In [5]:
# --- Find where the cold state file needs to go
# Cold state path & name
coldstate_path = read_from_control(controlFile,'settings_summa_path')
coldstate_name = read_from_control(controlFile,'settings_summa_coldstate')

# Specify default path if needed
if coldstate_path == 'default':
    coldstate_path = make_default_path('settings/SUMMA', controlFile) # outputs a Path()
else:
    coldstate_path = Path(coldstate_path) # make sure a user-specified path is a Path()
    
# Make the folder if it doesn't exist
coldstate_path.mkdir(parents=True, exist_ok=True)

In [6]:
# --- Find order and number of HRUs in forcing file
# Open the forcing file
forc = xr.open_dataset(forcing_path/forcing_name)

# Get the sorting order from the forcing file
forcing_hruIds = forc['hruId'].values.astype(int) # 'hruId' is prescribed by SUMMA so this variable must exist

# Number of HRUs
num_hru = len(forcing_hruIds)

3

In [7]:
# --- Define the dimensions and fill values
# Specify the dimensions
nSoil   = 1         # number of soil layers
nSnow   = 0         # assume no snow layers currently exist
midSoil = 1         # midpoint of soil layer
midToto = 1         # total number of midpoints for snow+soil layers
ifcToto = midToto+1 # total number of layer boundaries
scalarv = 1         # auxiliary dimension variable

In [8]:
# Time step size
dt_init = read_from_control(controlFile,'forcing_time_step_size') 

# Layer variables
mLayerDepth = np.asarray([4])
# mLayerDepth  = np.asarray([0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1, 1.5])
iLayerHeight = np.asarray([0,4])
# iLayerHeight = np.asarray([0, 0.025, 0.1, 0.25, 0.5, 1, 1.5, 2.5, 4])


In [9]:
# States
scalarCanopyIce      = 0      # Current ice storage in the canopy
scalarCanopyLiq      = 0      # Current liquid water storage in the canopy
scalarSnowDepth      = 0      # Current snow depth
scalarSWE            = 0      # Current snow water equivalent
scalarSfcMeltPond    = 0      # Current ponded melt water
scalarAquiferStorage = 1.0    # Current aquifer storage
scalarSnowAlbedo     = 0      # Snow albedo
scalarCanairTemp     = 283.16 # Current temperature in the canopy airspace
scalarCanopyTemp     = 283.16 # Current temperature of the canopy 
mLayerTemp           = 283.16 # Current temperature of each layer; assumed that all layers are identical
mLayerVolFracIce     = 0      # Current ice storage in each layer; assumed that all layers are identical
mLayerVolFracLiq     = 0.2    # Current liquid water storage in each layer; assumed that all layers are identical
mLayerMatricHead     = -1.0   # Current matric head in each layer; assumed that all layers are identical

In [10]:
# --- Make the initial conditions files
# auxiliary function used by the block that creates the .nc file
def create_and_fill_nc_var(nc, newVarName, newVarVal, fillDim1, fillDim2, newVarDim, newVarType, fillVal):
    
    # Make the fill value
    if newVarName == 'iLayerHeight' or newVarName == 'mLayerDepth':
        fillWithThis = np.full((fillDim1,fillDim2), newVarVal).transpose()
    else:
        fillWithThis = np.full((fillDim1,fillDim2), newVarVal)
    
    # Make the variable in the file
    ncvar = nc.createVariable(newVarName, newVarType, (newVarDim, 'hru',),fill_value=fillVal)        
    
    # Fill the variable
    ncvar[:] = fillWithThis
    
    return  

In [11]:
# --- Make the initial conditions files
# auxiliary function used by the block that creates the .nc file
def create_and_fill_nc_var(nc, newVarName, newVarVal, fillDim1, fillDim2, newVarDim, newVarType, fillVal):
    
    # Make the fill value
    if newVarName == 'iLayerHeight' or newVarName == 'mLayerDepth':
        fillWithThis = np.full((fillDim1,fillDim2), newVarVal).transpose()
    else:
        fillWithThis = np.full((fillDim1,fillDim2), newVarVal)
    
    # Make the variable in the file
    ncvar = nc.createVariable(newVarName, newVarType, (newVarDim, 'hru',),fill_value=fillVal)        
    
    # Fill the variable
    ncvar[:] = fillWithThis
    
    return  
    
# Create the empty trial params file
with nc4.Dataset(coldstate_path/coldstate_name, "w", format="NETCDF4") as cs:
    
    # === Some general attributes
    now = datetime.now()
    cs.setncattr('Author', "Created by SUMMA workflow scripts")
    cs.setncattr('History','Created ' + now.strftime('%Y/%m/%d %H:%M:%S'))
    cs.setncattr('Purpose','Create a cold state .nc file for initial SUMMA runs')
    
    # === Define the dimensions 
    cs.createDimension('hru',num_hru)
    cs.createDimension('midSoil',midSoil)
    cs.createDimension('midToto',midToto)
    cs.createDimension('ifcToto',ifcToto)
    cs.createDimension('scalarv',scalarv)
    
    # === Variables ===
    var = 'hruId'
    cs.createVariable(var, 'i4', 'hru', fill_value = False)
    cs[var].setncattr('units', '-')
    cs[var].setncattr('long_name', 'Index of hydrological response unit (HRU)')
    cs[var][:] = forcing_hruIds
    
    # time step size
    create_and_fill_nc_var(cs, 'dt_init', dt_init, 1, num_hru, 'scalarv', 'f8', False)
    
    # Number of layers
    create_and_fill_nc_var(cs, 'nSoil', nSoil, 1, num_hru, 'scalarv', 'i4', False)
    create_and_fill_nc_var(cs, 'nSnow', nSnow, 1, num_hru, 'scalarv', 'i4', False)
    
    # States
    create_and_fill_nc_var(cs, 'scalarCanopyIce',      scalarCanopyIce,      1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarCanopyLiq',      scalarCanopyLiq,      1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarSnowDepth',      scalarSnowDepth,      1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarSWE',            scalarSWE,            1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarSfcMeltPond',    scalarSfcMeltPond,    1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarAquiferStorage', scalarAquiferStorage, 1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarSnowAlbedo',     scalarSnowAlbedo,     1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarCanairTemp',     scalarCanairTemp,     1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'scalarCanopyTemp',     scalarCanopyTemp,     1,       num_hru, 'scalarv', 'f8', False)
    create_and_fill_nc_var(cs, 'mLayerTemp',           mLayerTemp,           midToto, num_hru, 'midToto', 'f8', False)
    create_and_fill_nc_var(cs, 'mLayerVolFracIce',     mLayerVolFracIce,     midToto, num_hru, 'midToto', 'f8', False)
    create_and_fill_nc_var(cs, 'mLayerVolFracLiq',     mLayerVolFracLiq,     midToto, num_hru, 'midToto', 'f8', False)
    create_and_fill_nc_var(cs, 'mLayerMatricHead',     mLayerMatricHead,     midSoil, num_hru, 'midSoil', 'f8', False)
    
    # layer dimensions
    create_and_fill_nc_var(cs, 'iLayerHeight', iLayerHeight, num_hru, ifcToto, 'ifcToto', 'f8', False)
    create_and_fill_nc_var(cs, 'mLayerDepth',  mLayerDepth,  num_hru, midToto, 'midToto', 'f8', False)
    
    

In [12]:
coldstate_path

PosixPath('/storage/dlhogan/summa_modeling_data/domain_EastRiver/settings/SUMMA')